In [1]:
!pip install SPARQLWrapper pandas
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.9/566.9 kB 9.1 MB/s eta 0:00:00:00:01


In [2]:
#Query DBLP - ECAI DOIs by year
sparql = SPARQLWrapper("https://sparql.dblp.org/sparql")
year_doi={}
years=range(1991,2025)
print(years)
print(len(years))
for year in years:
    year=int(year)
    query = f"""
    PREFIX dblp: <https://dblp.org/rdf/schema#>
    PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
    
    SELECT DISTINCT ?doi
    WHERE {{
      ?s dblp:publishedInStream <https://dblp.org/streams/conf/ecai> ;
             dblp:yearOfPublication "{year}"^^xsd:gYear ;
             dblp:doi ?doi .
    }}
    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    for r in results["results"]["bindings"]:
        if r:
            year_doi[year]=[]
    
    for r in results["results"]["bindings"]:
        doi = r["doi"]["value"]
        
        year_doi[year].append(doi)

range(1991, 2025)
34


In [ ]:
#Query SemOpenAlex - country's authors
import json
sparql = SPARQLWrapper("https://semopenalex.org/sparql")
countries_set=set()
countries_list=[]
for year in year_doi.keys():
    doi_list=year_doi[year]
    for doi in doi_list:
        query =f"""
            SELECT DISTINCT ?country WHERE {{
              ?work <http://purl.org/spar/datacite/doi> "{doi}" .
              ?work <http://purl.org/dc/terms/creator> ?author .
              ?author a <https://semopenalex.org/ontology/Author> .
              ?author <http://www.w3.org/ns/org#memberOf> ?aff .
              ?aff <https://dbpedia.org/ontology/location> ?loc .
              ?loc <https://dbpedia.org/property/country> ?country .
            }}
        """
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
    
        results = sparql.query().convert()
        for r in results["results"]["bindings"]:
            country   = r.get("country", {}).get("value")
            countries_set.add(country)
            countries_list.append(country)
dict_countries={}
for country in countries_set:
    dict_countries[country]=0

for country in countries_list:
    if country in dict_countries.keys():
        dict_countries[country]+=1
print(dict_countries)
output_path = "/kaggle/working/countries.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(dict_countries, f, ensure_ascii=False, indent=4)
    

In [5]:
#Query SemOpenAlex - all topics
sparql = SPARQLWrapper("https://semopenalex.org/sparql")
topics=set()
for year in year_doi.keys():
    doi_list=year_doi[year]
    for doi in doi_list:
        query = f"""
        SELECT DISTINCT * WHERE {{
          ?sub <http://purl.org/spar/datacite/doi> "{doi}" .
          ?sub <https://semopenalex.org/ontology/hasTopic> ?topic .
                                }}
                    """        
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
    
        results = sparql.query().convert()
        for r in results["results"]["bindings"]:
            topic   = r.get("topic", {}).get("value")
            topics.add(topic)

output_path = "/kaggle/working/topics.txt"
with open(output_path, "w", encoding="utf-8") as f:
    for topic in topics:
        f.write(topic+'\n')

#Extract label's topic
topic_name={}
for topic in topics:
    query = f"""
        SELECT DISTINCT ?label 
        WHERE {{
          <{topic}> <http://www.w3.org/2004/02/skos/core#prefLabel> ?label . 
                                }}
                    """        
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    results = sparql.query().convert()
    for r in results["results"]["bindings"]:
        label   = r.get("label", {}).get("value")
        topic_name[topic]=label
output_path = "/kaggle/working/topic_label.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(topic_name, f, ensure_ascii=False, indent=4)
import json
output_path = "/kaggle/working/topic_label.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(topic_name, f, ensure_ascii=False, indent=4)
            

KeyboardInterrupt: 

In [ ]:
#topics for each year
year_topics={}
for year in year_doi.keys():
    year_topics[year]=[]
    doi_list=year_doi[year]
    
    for doi in doi_list:
        query = f"""
        SELECT DISTINCT * WHERE {{
          ?sub <http://purl.org/spar/datacite/doi> "{doi}" .
          ?sub <https://semopenalex.org/ontology/hasTopic> ?topic .
                                }}
                    """        
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
    
        results = sparql.query().convert()
        for r in results["results"]["bindings"]:
            topic   = r.get("topic", {}).get("value")

In [ ]:
#Occurrences of topics by year
import numpy as np
from collections import Counter
import json

topic_year_counts={}
for year in year_topics.keys():
    topic_year_counts[year]={}

for year in year_topics.keys():
    year_topic=year_topics[year]
    topic_dict={}
    for topic in topics:
        topic_dict[topic]=0
    for topic in year_topic:
        if topic in topic_dict:
            topic_dict[topic]+=1
    topic_year_counts[year]=topic_dict
    print(topic_dict)
print(topic_year_counts)
output_path = "/kaggle/working/topic_year_counts.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(topic_year_counts, f, ensure_ascii=False, indent=4)